<a href="https://colab.research.google.com/github/yudits10/UAS_2_SearchEngine/blob/main/UAS_2_SearchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import re
import operator
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
news = pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/newsgroups.json')

In [4]:
news

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [5]:
news.head

<bound method NDFrame.head of                                                  content  target  \
0      From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1      From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2      From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3      From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4      From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   
...                                                  ...     ...   
11309  From: jim.zisfein@factory.com (Jim Zisfein) \n...      13   
11310  From: ebodin@pearl.tufts.edu\nSubject: Screen ...       4   
11311  From: westes@netcom.com (Will Estes)\nSubject:...       3   
11312  From: steve@hcrlgw (Steven Collins)\nSubject: ...       1   
11313  From: gunning@cco.caltech.edu (Kevin J. Gunnin...       8   

                   target_names  
0                     rec.autos  
1         comp.sys.mac.hardware  
2         comp.sys.mac.hardware  
3                

In [6]:
for i,txt in enumerate(news['content']):
    #print(i)
    subject = re.findall('Subject:(.*\n)',txt)
    if (len(subject) !=0):
        news.loc[i,'Subject'] =str(i)+' '+subject[0]
    else:
        news.loc[i,'Subject'] ='NA'

In [7]:
df_news =news[['Subject','content']]

In [8]:
df_news.head(5)

,Subject,content
0,0 WHAT car is this!?\n,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,1 SI Clock Poll - Final Call\n,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,2 PB questions...\n,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,3 Re: Weitek P9000 ?\n,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,4 Re: Shuttle Launch Question\n,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [9]:
df_news['content']=[entry.lower() for entry in df_news['content']]

<ipython-input-9-e54f969b1a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['content']=[entry.lower() for entry in df_news['content']]


In [10]:
SW=['subject:','organization:','thanks','thank','re:']
for sw in SW:
    df_news.content=df_news.content.replace(to_replace=sw,value='',regex=True)

<ipython-input-10-d7d28fb049ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content=df_news.content.replace(to_replace=sw,value='',regex=True)


In [11]:
df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True)
df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_news.content =df_news.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\]^_`{|}~]',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='-',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='\s+',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='  ',value='',regex=True)
df_news.content =df_news.content.apply(lambda x:x.strip())

<ipython-input-11-abc5a09876ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True)
<ipython-input-11-abc5a09876ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
<ipython-input-11-abc5a09876ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [12]:
df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',value=' ',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='\s+',value=' ',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='  ',value='',regex=True)
df_news.Subject =df_news.Subject.apply(lambda x:x.strip())

<ipython-input-12-e43feec3a853>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
<ipython-input-12-e43feec3a853>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',value=' ',regex=True)
<ipython-input-12-e43feec3a853>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [13]:
for i,sen in enumerate(df_news.content):
    if len(sen.strip()) ==0:
        print(str(i))
        #file_data.text[i] = np.nan
        df_news=df_news.drop(str(i),axis=0).reset_index().drop('index',axis=1)

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]

<ipython-input-15-bf0d29b469cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]


In [16]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):

        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
df_news.shape

(11314, 3)

In [18]:
df_news.head(2)

,Subject,content,Word tokenize
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,..."


In [19]:
df_clean = wordLemmatizer(df_news['Word tokenize'][0:1000])
df_clean

,Keyword_final
0,"['car', 'nntp', 'post', 'host', 'university', ..."
1,"['si', 'clock', 'poll', 'final', 'call', 'summ..."
2,"['pb', 'question', 'purdue', 'university', 'en..."
3,"['weitek', 'harris', 'computer', 'system', 'di..."
4,"['shuttle', 'launch', 'question', 'smithsonian..."
...,...
995,"['terminal', 'sale', 'nyx', 'public', 'access'..."
996,"['remember', 'name', 'come', 'election', 'time..."
997,"['interest', 'adb', 'behaviour', 'nntp', 'post..."
998,"['request', 'information', 'essential', 'tremo..."


In [20]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [21]:
df =pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/WordLemmatize20NewsGroup.json')
df_news['Clean_Keyword'] =df['Clean_Keyword']

In [22]:
df_news.head(5)

,Subject,content,Word tokenize,Clean_Keyword
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac...","thing,car,nntp,post,host,university,maryland,c..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,...","guy,kuo,si,clock,poll,final,call,summary,final..."
2,2 PB questions,pb questions... purdue university engineering ...,"[pb, questions, ..., purdue, university, engin...","thomas,willis,pb,question,purdue,university,en..."
3,3 Weitek P9000,weitek p9000 harris computer systems division ...,"[weitek, p9000, harris, computer, systems, div...","joe,green,weitek,harris,computer,system,divisi..."
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...,"[shuttle, launch, question, smithsonian, astro...","jonathan,mcdowell,shuttle,launch,question,smit..."


In [23]:
df_news_save= df_news
df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_news_save

,Subject,content
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...
2,2 PB questions,pb questions... purdue university engineering ...
3,3 Weitek P9000,weitek p9000 harris computer systems division ...
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...
...,...,...
11309,11309 Migraines and scans,migraines and scans distribution world inventi...
11310,11310 Screen Death Mac Plus 512,screen death mac plus 512 tufts university med...
11311,11311 Mounting CPU Cooler in vertical case,mounting cpu cooler in vertical case mail grou...
11312,11312 Sphere from 4 points,sphere from 4 points central research lab. hit...


In [24]:
df_news_save.to_csv("df_news_index.csv", index=False, header=True)

In [25]:
df_news.Clean_Keyword[0]

'thing,car,nntp,post,host,university,maryland,college,park,line,wonder,anyone,could,enlighten,car,saw,day,sport,car,look,late,early,call,bricklin,door,really,small,addition,front,bumper,separate,rest,body,know,anyone,tellme,model,name,engine,spec,year,production,car,make,history,whatever,info,funky,look,car,please,mail,il,bring,neighborhood,lerxst'

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

vocabulary = set()

for doc in df_news.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

tfidf.fit(df_news.Clean_Keyword)

tfidf_tran=tfidf.transform(df_news.Clean_Keyword)


In [27]:
vocabulary[0:4]

['enig', 'nobles', 'jeo', 'spagthorpe']

In [28]:
with open('/content/tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [29]:
t = pickle.load(open('/content/tfid.pkl','rb'))

In [30]:
with open("/content/vocabulary_news20group.txt", "w") as file:
    file.write(str(vocabulary))

In [31]:
with open("/content/vocabulary_news20group.txt", "r") as file:
    data2 = eval(file.readline())

In [32]:
print(data2[0:10])

['enig', 'nobles', 'jeo', 'spagthorpe', 'saa', 'keywords', 'usenix', 'yale', 'misjudge', 'harro']


In [33]:
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))

    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [34]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [35]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    #print("\nQuery:", query)
    #print("")
    #print(tokens)

    d_cosines = []

    query_vector = gen_vector_T(q_df['q_clean'])

    for d in tfidf_tran.A:

        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_news['Subject'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [36]:
%time cosine_similarity_T(10,'migraines and scans')

CPU times: user 3.34 s, sys: 7.97 s, total: 11.3 s
Wall time: 11.1 s


,index,Subject,Score
0,1300,1300 migraine and exercise,0.493360
1,8465,8465 Exercise and Migraine,0.441951
2,6790,6790 various migraine therapies,0.437929
3,2921,2921 Exercise and Migraine,0.428249
4,5646,5646 Exercise and Migraine,0.412856
5,10251,10251 Migraines and Estrogen,0.311562
6,5389,5389 migraine and exercise,0.288223
7,272,272 Could this be a migraine,0.274159
8,357,357 Half page hand scanners,0.273738
9,11309,11309 Migraines and scans,0.239886


In [37]:
df_news.content[1300]

'migraine and exercise reply to geb cs.pitt.edu gordon banks univ. of pittsburgh computer science in article c4iozx.7wx world.std.com rsilver world.std.com richard silver writes i have two questions. is there any obvious connection between the flushed appearance and the migraine was i foolish to play through the migraine aside from the visual disturbance affecting my play i just prefer to ignore it when possible. the flushing is due to vascular dilation part of a migraine attack. some people event get puffy and swollen. as long as you are careful you can see well enough to avoid getting hit in the face or eye by the ball migraine will not hurt your health. gordon banks n3jxp skepticism is the chastity of the intellect and geb cadre.dsl.pitt.edu it is shameful to surrender it too soon.'